In [7]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import fsolve
import anndata as ad
import scanpy as sc
from PIL import Image
import tifffile as tifi
import squidpy as sq
from scipy.sparse import issparse

/Users/siyuanzhao/anaconda3/envs/VDS/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 14.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/Users/siyuanzhao/anaconda3/envs/VDS/lib/python3.10/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [8]:
hirescalef = 0.10757315

SAMPLES = {
    "skin_TXK6Z4X_A1": {
        "id": "skin_TXK6Z4X_A1",
        "name": "skin_TXK6Z4X_A1",
        "adata": "../Data/skin_TXK6Z4X_A1_processed/tmap/weighted_by_area|celltypist_cells_adata.h5",
        "wsi": "../Data/skin_TXK6Z4X_A1_processed/tmap/wsi.tif",
        "tiles": "../Data/skin_TXK6Z4X_A1_processed/skin_TXK6Z4X_A1_processed_tiles",
        "cells_layer": "../Data/skin_TXK6Z4X_A1_processed/cells_layer.png",
    },
    "skin_TXK6Z4X_D1": {
        "id": "skin_TXK6Z4X_D1",
        "name": "skin_TXK6Z4X_D1",
        "adata": "../Data/skin_TXK6Z4X_D1_processed/tmap/weighted_by_area|celltypist_cells_adata.h5",
        "wsi": "../Data/skin_TXK6Z4X_D1_processed/tmap/wsi.tif",
        "tiles": "../Data/skin_TXK6Z4X_D1_processed/skin_TXK6Z4X_D1_processed_tiles",
        "cells_layer": "../Data/skin_TXK6Z4X_D1_processed/cells_layer.png",
    },
}

In [9]:
def get_cell_type_coordinates(sample_id):
    if sample_id == "skin_TXK6Z4X_A1":
        adata = sc.read_h5ad(SAMPLES["skin_TXK6Z4X_A1"]["adata"])
    elif sample_id == "skin_TXK6Z4X_D1":
        adata = sc.read_h5ad(SAMPLES["skin_TXK6Z4X_D1"]["adata"])

    df = adata.obsm["spatial"].copy()
    df["cell_type"] = adata.obs["cell_type"]
    df["id"] = adata.obs.index
    return df

In [34]:
cell_list = ["ID_65655", "ID_65658", "ID_65659", "ID_70820"]
# cell_list = []
gene_list = ["COL1A1", "COL3A1"]
sample_ids = ["skin_TXK6Z4X_A1"]

In [32]:
def filter_and_merge(cell_ids, gene_names, sample_ids):
    results = {}
    for sample_id in sample_ids:
        if sample_id not in SAMPLES:
            raise ValueError("Sample not found.")
        else:
            adata_path = SAMPLES[sample_id]["adata"]
            adata = sc.read_h5ad(adata_path)

            if not cell_ids:
                valid_cell_ids = adata.obs_names.tolist()
            else:
                valid_cell_ids = [cell for cell in cell_ids if cell in adata.obs_names]

            valid_gene_names = [gene for gene in gene_names if gene in adata.var_names]

            if valid_gene_names:
                filtered_adata = adata[valid_cell_ids, valid_gene_names].copy()
                if issparse(filtered_adata.X):
                    expr_data = filtered_adata.X.toarray()
                else:
                    expr_data = filtered_adata.X
                expr_df = pd.DataFrame(
                    expr_data,
                    index=filtered_adata.obs_names,
                    columns=filtered_adata.var_names,
                )
            else:
                expr_df = pd.DataFrame(index=valid_cell_ids)

            expr_df = expr_df.reset_index().rename(columns={"index": "id"})

            missing_genes = [gene for gene in gene_names if gene not in expr_df.columns]
            for gene in missing_genes:
                expr_df[gene] = 0

            expr_df = expr_df[['id'] + gene_names]

            coord_df = get_cell_type_coordinates(sample_id).reset_index(drop=True)

            merged_df = pd.merge(expr_df, coord_df, on="id", how="inner")

            merged_df["total_expression"] = adata[valid_cell_ids, :].X.sum(axis=1)

            for gene in gene_names:
                merged_df[f"{gene}_ratio"] = np.where(
                    merged_df["total_expression"] == 0,
                    0,
                    merged_df[gene] / merged_df["total_expression"],
                )

            results[sample_id] = merged_df

    return results

In [18]:
def filter_and_cumsum(merged_df, selected_columns):
    def conditional_cumsum(row, selected_columns):
        cumsum = 0
        for col in selected_columns:
            if row[col] != 0:
                cumsum += row[col]
                row[col] = cumsum
        return row

    filtered_df = merged_df[["id", "cell_x", "cell_y"] + selected_columns].copy()
    filtered_df[selected_columns] = filtered_df[selected_columns].apply(
        conditional_cumsum, axis=1, selected_columns=selected_columns
    )
    return filtered_df

In [25]:
def get_kosara_data(sample_ids, gene_list, cell_list):
    radius = 5
    d = np.sqrt(2) * radius

    special_r, special_d = np.sqrt(2) * radius, np.sqrt(2) * radius
    special_angle1 = np.degrees(
        np.arccos((special_r**2 + d**2 - radius**2) / (2 * special_r * d))
    )
    special_angle2 = np.degrees(
        np.arccos((radius**2 + d**2 - special_r**2) / (2 * radius * d))
    )
    special_result = (
        special_angle1 * special_r**2
        + special_angle2 * radius**2
        - d * special_r * np.sin(special_angle1)
    )
    special_value = special_result / (np.pi * radius**2)

    def equation(r, d, a, radius_val, special_value):
        angle1 = np.arccos((r**2 + d**2 - radius_val**2) / (2 * r * d))
        angle2 = np.arccos((radius_val**2 + d**2 - r**2) / (2 * radius_val * d))
        if a <= special_value:
            result = angle1 * r**2 + angle2 * radius_val**2 - d * r * np.sin(angle1)
        else:
            result = (
                angle1 * r**2
                + angle2 * radius_val**2
                - r**2 * np.sin(angle1) * np.cos(angle1)
                - radius_val**2 * np.sin(angle2) * np.cos(angle2)
            )
        return result - (a * np.pi * radius_val**2)

    def initial_guess(a, d, special_value):
        if a <= special_value:
            return d + 0.01
        elif a > 0.95:
            return 8
        else:
            return a * 10 - 1.5

    def calculate_radius(originaldf, radius_val):
        # originaldf = originaldf.copy()
        originaldf["radius"] = radius_val
        result_df = originaldf.copy()
        # result_df = originaldf[["id", "cell_x", "cell_y", "radius"] + gene_list].copy()
        for index, row in originaldf.iterrows():
            for col in gene_list:
                a_value = row[col]
                if a_value == 0:
                    cal_radius = 0
                    angle = 0
                else:
                    try:
                        guess = initial_guess(a_value, d, special_value)
                        cal_radius = fsolve(
                            equation,
                            guess,
                            args=(d, a_value, radius_val, special_value),
                        )[0]
                        angle = np.degrees(
                            np.arccos(
                                (cal_radius**2 + d**2 - radius_val**2)
                                / (2 * cal_radius * d)
                            )
                        )
                    except ValueError as e:
                        cal_radius = np.nan
                        angle = np.nan
                result_df.loc[index, f"{col}_radius"] = cal_radius
                result_df.loc[index, f"{col}_angle"] = angle

        return result_df

    position_cell_ratios_dict = filter_and_merge(cell_list, gene_list, sample_ids)
    results = {}

    for sample_id, merged_df in position_cell_ratios_dict.items():
        # cumsum_df = filter_and_cumsum(merged_df, gene_list)

        kosara_df = calculate_radius(merged_df, radius)

        results[sample_id] = kosara_df.to_dict(orient="records")

    return results

In [35]:
position_cell_ratios_dict = filter_and_merge(cell_list, gene_list, sample_ids)
position_cell_ratios_dict["skin_TXK6Z4X_A1"]

,id,COL1A1,COL3A1,cell_x,cell_y,cell_type,total_expression,COL1A1_ratio,COL3A1_ratio
0,ID_65655,1,3,6335,7530,stromal 1,4,0.2500,0.750
1,ID_65658,1,0,5186,7986,cms2,1,1.0000,0.000
2,ID_65659,0,0,6850,7531,cms2,0,0.0000,0.000
3,ID_70820,15,50,7305,7474,myofibroblasts,80,0.1875,0.625


In [36]:
result = get_kosara_data(sample_ids, gene_list, cell_list)
result["skin_TXK6Z4X_A1"]

/var/folders/pm/_db3z8953vb1gtk99jkk_sb80000gn/T/ipykernel_68253/2659310979.py:20: RuntimeWarning: invalid value encountered in arccos
  angle1 = np.arccos((r**2 + d**2 - radius_val**2) / (2 * r * d))
/var/folders/pm/_db3z8953vb1gtk99jkk_sb80000gn/T/ipykernel_68253/2659310979.py:21: RuntimeWarning: invalid value encountered in arccos
  angle2 = np.arccos((radius_val**2 + d**2 - r**2) / (2 * radius_val * d))
/var/folders/pm/_db3z8953vb1gtk99jkk_sb80000gn/T/ipykernel_68253/2659310979.py:20: RuntimeWarning: invalid value encountered in arccos
  angle1 = np.arccos((r**2 + d**2 - radius_val**2) / (2 * r * d))
/var/folders/pm/_db3z8953vb1gtk99jkk_sb80000gn/T/ipykernel_68253/2659310979.py:21: RuntimeWarning: invalid value encountered in arccos
  angle2 = np.arccos((radius_val**2 + d**2 - r**2) / (2 * radius_val * d))
/var/folders/pm/_db3z8953vb1gtk99jkk_sb80000gn/T/ipykernel_68253/2659310979.py:55: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement fr

[{'id': 'ID_65655',
  'COL1A1': 1,
  'COL3A1': 3,
  'cell_x': 6335,
  'cell_y': 7530,
  'cell_type': 'stromal 1',
  'total_expression': 4,
  'COL1A1_ratio': 0.25,
  'COL3A1_ratio': 0.75,
  'radius': 5,
  'COL1A1_radius': 12.07106772789294,
  'COL1A1_angle': 0.005682977263453445,
  'COL3A1_radius': 7.081067811865475,
  'COL3A1_angle': 41.3789436621239},
 {'id': 'ID_65658',
  'COL1A1': 1,
  'COL3A1': 0,
  'cell_x': 5186,
  'cell_y': 7986,
  'cell_type': 'cms2',
  'total_expression': 1,
  'COL1A1_ratio': 1.0,
  'COL3A1_ratio': 0.0,
  'radius': 5,
  'COL1A1_radius': 12.07106772789294,
  'COL1A1_angle': 0.005682977263453445,
  'COL3A1_radius': 0.0,
  'COL3A1_angle': 0.0},
 {'id': 'ID_65659',
  'COL1A1': 0,
  'COL3A1': 0,
  'cell_x': 6850,
  'cell_y': 7531,
  'cell_type': 'cms2',
  'total_expression': 0,
  'COL1A1_ratio': 0.0,
  'COL3A1_ratio': 0.0,
  'radius': 5,
  'COL1A1_radius': 0.0,
  'COL1A1_angle': 0.0,
  'COL3A1_radius': 0.0,
  'COL3A1_angle': 0.0},
 {'id': 'ID_70820',
  'COL1A1': 15

In [38]:
result["skin_TXK6Z4X_A1"][3]

{'id': 'ID_70820',
 'COL1A1': 15,
 'COL3A1': 50,
 'cell_x': 7305,
 'cell_y': 7474,
 'cell_type': 'myofibroblasts',
 'total_expression': 80,
 'COL1A1_ratio': 0.1875,
 'COL3A1_ratio': 0.625,
 'radius': 5,
 'COL1A1_radius': 7.081067811865475,
 'COL1A1_angle': 41.3789436621239,
 'COL3A1_radius': 8.0,
 'COL3A1_angle': 38.125874240855254}